In [11]:
from numpy import append
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

class Scraper:

    def __init__(self, URL : str, driver : webdriver.Chrome):
        
        self.driver = driver
        self.driver_list = []
        self.team_list = []
        self.URL = URL
        time.sleep(1) 

    def load_and_accept_cookies(self) -> None:

        '''
        Open site and accept the cookies
        
        Returns
        -------
        driver: webdriver.Chrome
            This driver is already in the site webpage
        '''
        self.driver.get(self.URL)
        accept_button = self.driver.find_element(by=By.XPATH, value="//*[@class='btn btn-success acceptGdpr']")
        accept_button.click()
        


    def get_drivers(self):

        
        self.driver.get(self.URL)
        time.sleep(2)
        navbar = self.driver.find_element(by=By.XPATH, value="//div[@class='navbar-nav']")
        drivers_button= navbar.find_element(by=By.LINK_TEXT, value = 'Drivers')
        drivers_button.click()
        self.get_driver_data()
        
    def get_driver_data(self):
        
        starting_letter_tag = self.driver.find_elements(by=By.XPATH, value="//div[@class='col-sm-6 col-md-4']")

        for driver in starting_letter_tag:

            a_tag = driver.find_element(By.TAG_NAME, 'a')
            link = a_tag.get_attribute('href')
            self.driver_list.append(link)
        
        for link in self.driver_list:
            self.driver.get(link)
            nationality_column = driver.find_element(by=By.XPATH, value="//div[@class='col-md-6']")
            nationality_row = nationality_column.find_elements(By.TAG_NAME, 'td')
            print(nationality_row[1].text)

    def get_teams(self):

        self.driver.get(self.URL)
        time.sleep(1) 
        time.sleep(2)
        navbar = self.driver.find_element(by=By.XPATH, value="//div[@class='navbar-nav']")
        teams_button= navbar.find_element(by=By.LINK_TEXT, value = 'Teams')
        teams_button.click()
        
    def get_champs(self):

        self.driver.get(self.URL)
        time.sleep(1) 
        time.sleep(2)
        navbar = self.driver.find_element(by=By.XPATH, value="//div[@class='navbar-nav']")
        champs_button= navbar.find_element(by=By.LINK_TEXT, value = 'Champions')
        champs_button.click()

if __name__ == "__main__":
    scraper = Scraper("https://www.4mula1stats.com/", webdriver.Chrome())
    scraper.load_and_accept_cookies()
    scraper.get_drivers()
    #scraper.get_teams()
    #scraper.get_champs()
    

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=103.0.5060.114)
Stacktrace:
#0 0x557ef01a9b13 <unknown>
#1 0x557eeffb0688 <unknown>
#2 0x557eeffb34e7 <unknown>
#3 0x557eeffb338b <unknown>
#4 0x557eeffb364c <unknown>
#5 0x557eeffe79ee <unknown>
#6 0x557eeffe7e91 <unknown>
#7 0x557eeffdc7d7 <unknown>
#8 0x557ef00058dd <unknown>
#9 0x557eeffdc713 <unknown>
#10 0x557ef00059be <unknown>
#11 0x557ef0018b94 <unknown>
#12 0x557ef00057a3 <unknown>
#13 0x557eeffdb0ea <unknown>
#14 0x557eeffdc225 <unknown>
#15 0x557ef01f12dd <unknown>
#16 0x557ef01f52c7 <unknown>
#17 0x557ef01db22e <unknown>
#18 0x557ef01f60a8 <unknown>
#19 0x557ef01cfbc0 <unknown>
#20 0x557ef02126c8 <unknown>
#21 0x557ef0212848 <unknown>
#22 0x557ef022cc0d <unknown>
#23 0x7f063ec03b43 <unknown>
